In [37]:
import os
import requests
import json
from calendar import month_name
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import date, datetime

import pandas as pd
from IPython.display import display, clear_output
from deltalake import DeltaTable
from deltalake.exceptions import TableNotFoundError
from tenacity import retry, retry_if_exception, stop_after_attempt, wait_exponential


In [38]:
token = notebookutils.credentials.getToken("pbi")

## ⚠️ Parameters to look at 

In [39]:
# In case you only want to run for a subset of your accounts, please mention them here.
# By default it will run against all your accounts.
# if set, must exactly match the TPAccountName seen in MSXI reports 
# accounts = ["'FOD BOSA'", "'Vlaamse Overheid'", "'Eurocontrol'", "'bpost'", "'Nationale Loterij/Loterie Nationale'", "'Region Bruxelles Capitale'"]

# whether to only update incrementally (reload only current and previous month) or do a full load (Previous FY + FY until now)
table_path = f"{notebookutils.lakehouse.get('LH_acr').properties['abfsPath']}/Tables/acr"
storage_options = {"bearer_token": notebookutils.credentials.getToken('storage'), "use_fabric_endpoint": "true"}
try:
    DeltaTable(table_path, storage_options=storage_options)
    incremental = True
except TableNotFoundError:
    incremental = False
print(f"Incremental load set to: {incremental}")

Incremental load set to: True


## Accounts

In [40]:
if "accounts" not in globals():
    url = "https://df-msit-scus-redirect.analysis.windows.net/explore/querydata?synchronous=true"
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7",
        "activityid": "0d875846-54e5-4ed1-aac1-4f4bef9dfb7e",
        "authorization": f"Bearer {token}",
        "content-type": "application/json;charset=UTF-8",
        "origin": "https://msit.powerbi.com",
        "priority": "u=1, i",
        "referer": "https://msit.powerbi.com/",
        "requestid": "1aba8c14-33f0-4ed9-0623-999708b6d67f",
        "sec-ch-ua": '"Not;A=Brand";v="99", "Microsoft Edge";v="139", "Chromium";v="139"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36 Edg/139.0.0.0",
        "x-powerbi-hostenv": "Power BI Web App"
    }
    query = {"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"d2","Entity":"DimDate","Type":0},{"Name":"m","Entity":"M_ACR","Type":0},{"Name":"d","Entity":"DimCustomer","Type":0},{"Name":"d1","Entity":"Distinct Enrollment","Type":0},{"Name":"a","Entity":"ACR_Parameters","Type":0},{"Name":"f","Entity":"Field Hierarchy","Type":0},{"Name":"d3","Entity":"Distinct AdjustmentFlag","Type":0},{"Name":"d11","Entity":"DimViewType","Type":0},{"Name":"d4","Entity":"DimAccountSummary","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalMonth"},"Name":"DimDate.FiscalMonth","NativeReferenceName":"FiscalMonth"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"$ Organic ACR"},"Name":"M_ACR.$ Organic ACR","NativeReferenceName":"$ Organic ACR"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"$ Average Daily Organic ACR"},"Name":"M_ACR.$ Average Daily Organic ACR","NativeReferenceName":"$ Average Daily Organic ACR"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"$ Avg Daily Organic ACR MoM Change"},"Name":"M_ACR.$ Avg Daily Organic ACR MoM Change","NativeReferenceName":"$ Avg Daily Organic ACR MoM Change"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"% Avg Daily Organic ACR MoM"},"Name":"M_ACR.% Avg Daily Organic ACR MoM","NativeReferenceName":"% Avg Daily Organic ACR MoM"},{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"TPAccountName"},"Name":"DimCustomer.TPAccountName","NativeReferenceName":"TPAccountName"},{"Column":{"Expression":{"SourceRef":{"Source":"d1"}},"Property":"EnrollmentCustomerName"},"Name":"Distinct Enrollment.EnrollmentCustomerName","NativeReferenceName":"EnrollmentCustomerName"}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"a"}},"Property":"ACR_Parameters Fields"}}],"Values":[[{"Literal":{"Value":"\'\'\'Distinct SubscriptionGUID\'\'[SubscriptionGUID]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct ServiceLevel4\'\'[ServiceLevel1]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct ServiceLevel4\'\'[ServiceLevel2]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct SubscriptionGUID\'\'[SubscriptionName]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct Enrollment\'\'[EnrollmentCustomerName]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct SubStrategicPillar\'\'[SuperStrategicPillar]\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"f"}},"Property":"Field Hierarchy Fields"}}],"Values":[[{"Literal":{"Value":"\'\'\'DimCustomer\'\'[TPAccountName]\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d3"}},"Property":"AdjustmentFlag"}}],"Values":[[{"Literal":{"Value":"\'N/A\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d11"}},"Property":"ViewType"}}],"Values":[[{"Literal":{"Value":"\'Curated\'"}}]]}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"HQ_DS"}}],"Values":[[{"Literal":{"Value":"\'DS\'"}}]]}}}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FY_Rel"}}],"Values":[[{"Literal":{"Value":"\'FY\'"}}],[{"Literal":{"Value":"\'FY+1\'"}}],[{"Literal":{"Value":"\'FY-1\'"}}]]}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"SegmentGroup"}}],"Values":[[{"Literal":{"Value":"null"}}]]}}}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d4"}},"Property":"Segment"}}],"Values":[[{"Literal":{"Value":"null"}}],[{"Literal":{"Value":"\'SME&C - SMB Commercial\'"}}]]}}}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3,4,5,6],"Subtotal":1}]},"DataReduction":{"Primary":{"Top":{"Count":1000000}},"Secondary":{"Top":{"Count":100}}},"Version":1}}}]},"QueryId":"6d5da4b7-9e70-2d90-e0dc-db2828a92d8f"}],"cancelQueries":[],"modelId":6232338,"userPreferredLocale":"en-US"}
    response = requests.post(url, headers=headers, json=query)
    response.raise_for_status()
    mapping_col_group = {item["Name"].split(".", 1)[-1]: item["Value"] for item in response.json()["results"][0]["result"]["data"]["descriptor"]["Select"] if "G" in item["Value"]}
    assert "TPAccountName" in mapping_col_group, "Could not find the mapping for TPAccountName, thus cannot find customers"
    group = mapping_col_group["TPAccountName"]
    mapping_group_to_dn = {item["N"]: item.get("DN") for item in response.json()["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][1]["DM1"][0]["S"]}
    dn = mapping_group_to_dn.get(group)
    accounts = response.json()["results"][0]["result"]["data"]["dsr"]["DS"][0]["ValueDicts"][dn]

print("⚠️ Script will run for the following accounts:\n- " + '\n- '.join(accounts))
accounts = ["'" + account.replace("'", "''") + "'" for account in accounts]

⚠️ Script will run for the following accounts:
- bpost
- Eurocontrol
- FOD BOSA
- Nationale Loterij/Loterie Nationale
- Region Bruxelles Capitale
- Vlaamse Overheid


## CSV based extraction
Here no DLP policy can be applied however, limited to 30 000 records.
The result from the endpoint is some strange data format called DSR; which is native to Power BI / Analysis services.
Luckily someone on the internet created a parser: https://gist.github.com/svavassori/3319ff9d7e16a8788665ca59a5a04889

In [41]:
url = "https://df-msit-scus-redirect.analysis.windows.net/explore/querydata?synchronous=true"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7",
    "activityid": "0d875846-54e5-4ed1-aac1-4f4bef9dfb7e",
    "authorization": f"Bearer {token}",
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://msit.powerbi.com",
    "priority": "u=1, i",
    "referer": "https://msit.powerbi.com/",
    "requestid": "1aba8c14-33f0-4ed9-0623-999708b6d67f",
    "sec-ch-ua": '"Not;A=Brand";v="99", "Microsoft Edge";v="139", "Chromium";v="139"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36 Edg/139.0.0.0",
    "x-powerbi-hostenv": "Power BI Web App"
}

query = {"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"d2","Entity":"DimDate","Type":0},{"Name":"m","Entity":"M_ACR","Type":0},{"Name":"d","Entity":"DimCustomer","Type":0},{"Name":"d1","Entity":"Distinct SubscriptionGUID","Type":0},{"Name":"d4","Entity":"Distinct Enrollment","Type":0},{"Name":"d3","Entity":"Distinct ServiceLevel4","Type":0},{"Name":"a","Entity":"ACR_Parameters","Type":0},{"Name":"f","Entity":"Field Hierarchy","Type":0},{"Name":"d21","Entity":"Distinct AdjustmentFlag","Type":0},{"Name":"d31","Entity":"DimViewType","Type":0},{"Name":"d41","Entity":"DimAccountSummary","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalMonth"},"Name":"DimDate.FiscalMonth","NativeReferenceName":"FiscalMonth"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"$ Organic ACR"},"Name":"M_ACR.$ Organic ACR","NativeReferenceName":"$ Organic ACR"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"$ Average Daily Organic ACR"},"Name":"M_ACR.$ Average Daily Organic ACR","NativeReferenceName":"$ Average Daily Organic ACR"},{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"TPAccountName"},"Name":"DimCustomer.TPAccountName","NativeReferenceName":"TPAccountName"},{"Column":{"Expression":{"SourceRef":{"Source":"d1"}},"Property":"SubscriptionGUID"},"Name":"Distinct SubscriptionGUID.SubscriptionGUID","NativeReferenceName":"SubscriptionGUID"},{"Column":{"Expression":{"SourceRef":{"Source":"d1"}},"Property":"SubscriptionName"},"Name":"Distinct SubscriptionGUID.SubscriptionName","NativeReferenceName":"SubscriptionName"},{"Column":{"Expression":{"SourceRef":{"Source":"d4"}},"Property":"EnrollmentCustomerName"},"Name":"Distinct Enrollment.EnrollmentCustomerName","NativeReferenceName":"EnrollmentCustomerName"},{"Column":{"Expression":{"SourceRef":{"Source":"d3"}},"Property":"ServiceLevel1"},"Name":"Distinct ServiceLevel4.ServiceLevel1","NativeReferenceName":"ServiceLevel1"},{"Column":{"Expression":{"SourceRef":{"Source":"d3"}},"Property":"ServiceLevel2"},"Name":"Distinct ServiceLevel4.ServiceLevel2","NativeReferenceName":"ServiceLevel2"},{"Column":{"Expression":{"SourceRef":{"Source":"d3"}},"Property":"ServiceLevel4"},"Name":"Distinct ServiceLevel4.ServiceLevel4","NativeReferenceName":"ServiceLevel4"},{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalYear"},"Name":"DimDate.FiscalYear","NativeReferenceName":"FiscalYear"}],"Where":[{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalYear"}},{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalQuarter"}},{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FiscalMonthName"}}],"Values":[[{"Literal":{"Value":"\'FY25\'"}},{"Literal":{"Value":"\'FY25-Q4\'"}},{"Literal":{"Value":"\'May 2025\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"TPAccountName"}}],"Values":[[{"Literal":{"Value":"\'bpost\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"a"}},"Property":"ACR_Parameters Fields"}}],"Values":[[{"Literal":{"Value":"\'\'\'Distinct SubscriptionGUID\'\'[SubscriptionGUID]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct ServiceLevel4\'\'[ServiceLevel1]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct ServiceLevel4\'\'[ServiceLevel2]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct SubscriptionGUID\'\'[SubscriptionName]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct Enrollment\'\'[EnrollmentCustomerName]\'"}}],[{"Literal":{"Value":"\'\'\'Distinct SubStrategicPillar\'\'[SuperStrategicPillar]\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"f"}},"Property":"Field Hierarchy Fields"}}],"Values":[[{"Literal":{"Value":"\'\'\'DimCustomer\'\'[TPAccountName]\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d21"}},"Property":"AdjustmentFlag"}}],"Values":[[{"Literal":{"Value":"\'N/A\'"}}]]}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d31"}},"Property":"ViewType"}}],"Values":[[{"Literal":{"Value":"\'Curated\'"}}]]}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"HQ_DS"}}],"Values":[[{"Literal":{"Value":"\'DS\'"}}]]}}}}},{"Condition":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d2"}},"Property":"FY_Rel"}}],"Values":[[{"Literal":{"Value":"\'FY\'"}}],[{"Literal":{"Value":"\'FY+1\'"}}],[{"Literal":{"Value":"\'FY-1\'"}}]]}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d"}},"Property":"SegmentGroup"}}],"Values":[[{"Literal":{"Value":"null"}}]]}}}}},{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"d41"}},"Property":"Segment"}}],"Values":[[{"Literal":{"Value":"null"}}],[{"Literal":{"Value":"\'SME&C - SMB Commercial\'"}}]]}}}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3,4,5,6,7,8,9,10],"Subtotal":1}]},"DataReduction":{"Primary":{"Top":{"Count":1000000}},"Secondary":{"Top":{"Count":100}}},"Version":1}}}]},"QueryId":"fd42d142-99cd-e036-e59b-b6dd9a67ad27"}],"cancelQueries":[],"modelId":6232338,"userPreferredLocale":"en-US"}
account = "'Eurocontrol'"
query["queries"][0]["Query"]["Commands"][0]["SemanticQueryDataShapeCommand"]["Query"]["Where"][1]["Condition"]["In"]["Values"][0][0]["Literal"]["Value"] = account

# response = requests.post(url, headers=headers, json=query)
# response

In [42]:
# code slightly vibe code modified from: https://gist.github.com/svavassori/3319ff9d7e16a8788665ca59a5a04889
def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def extract_dm1_data(response):
    """
    Extract DM1 data from PowerBI JSON export.
    This is the detailed breakdown data you actually want.
    """
    data = response.json()

    # Navigate to data
    result_data = data["results"][0]["result"]["data"]
    dsr_data = result_data["dsr"]["DS"][0]
    value_dicts = dsr_data.get("ValueDicts", {})

    # Find DM1 data (detailed information, DM0 is high level aggregate)
    # this differs from the original script
    if len(dsr_data["PH"]) < 2:
        raise ValueError("Only one data result in query, manual check needed")

    if "DM1" not in dsr_data["PH"][1]:
        raise ValueError("No DM1 data found")

    dm1_data = dsr_data["PH"][1]["DM1"]
    
    if not dm1_data: # this is possible for example for a new month or if you would do a query for the future
        return None

    # Get column structure
    columns_info = dm1_data[0]["S"]

    # Get column names from descriptor (only take after the '.' as actual column name)
    descriptor = result_data["descriptor"]["Select"]
    group_cols = [col["Name"].split(".", 1)[1] for col in descriptor if col["Kind"] == 1]
    measure_cols = [col["Name"].split(".", 1)[1] for col in descriptor if col["Kind"] == 2]
    all_columns = group_cols + measure_cols

    # Process each row
    rows = []
    prev_row = None

    for item in dm1_data:
        current = item.get("C", []).copy()

        # Handle R bitfield (reuse previous values)
        if "R" in item and prev_row:
            r_bits = item["R"]
            new_row = []
            curr_idx = 0

            for col_idx in range(len(columns_info)):
                if (r_bits >> col_idx) & 1:  # Bit is set - reuse
                    new_row.append(prev_row[col_idx])
                else:  # Use current value
                    if curr_idx < len(current):
                        new_row.append(current[curr_idx])
                        curr_idx += 1
                    else:
                        new_row.append(None)
            current = new_row

        # Handle null bitfield
        if "Ø" in item:
            null_bits = item["Ø"]
            for i in range(len(current)):
                if (null_bits >> i) & 1:
                    current.insert(i, None)

        # Expand dictionary values
        for col_idx, col_info in enumerate(columns_info):
            if col_idx < len(current) and "DN" in col_info:
                value = current[col_idx]
                if isinstance(value, int):
                    dict_key = col_info["DN"]
                    if dict_key in value_dicts and 0 <= value < len(value_dicts[dict_key]):
                        current[col_idx] = value_dicts[dict_key][value]
        
        current = [round(float(i), 4) if isinstance(i, str) and is_number(i) else i for i in current]
        rows.append(current)
        prev_row = current
    
    df = pd.DataFrame(rows, columns=all_columns[: len(rows[0]) if rows else len(all_columns)])
    return df

In [43]:
def parse_fiscal_month(fiscal_month_str):
    """
    Parse fiscal month string (e.g., 'FY25-Jun') to actual date.
    
    Fiscal year starts in July, so:
    - FY25 runs from July 2024 to June 2025
    - FY26 runs from July 2025 to June 2026
    
    Args:
        fiscal_month_str (str): String in format 'FYxx-Mon'
    
    Returns:
        datetime: First day of the actual calendar month
    """
    # Split the string into fiscal year and month
    parts = fiscal_month_str.split('-')
    if len(parts) != 2:
        raise ValueError(f"Invalid format: {fiscal_month_str}")
    
    fy_part = parts[0]
    month_abbr = parts[1]
    
    # Extract fiscal year number (e.g., 25 from FY25)
    if not fy_part.startswith('FY'):
        raise ValueError(f"Invalid fiscal year format: {fy_part}")
    
    fy_num = int(fy_part[2:])
    
    # Convert to full year (assuming 2000s/2100s based on value)
    if fy_num < 100:
        fiscal_year = 2000 + fy_num
    else:
        fiscal_year = fy_num
    
    # Map month abbreviation to month number
    month_map = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    if month_abbr not in month_map:
        raise ValueError(f"Invalid month abbreviation: {month_abbr}")
    
    month_num = month_map[month_abbr]
    
    # Determine actual calendar year
    # Fiscal year starts in July (month 7)
    # July-Dec of FY25 = calendar year 2024
    # Jan-Jun of FY25 = calendar year 2025
    if month_num >= 7:  # July to December
        calendar_year = fiscal_year - 1
    else:  # January to June
        calendar_year = fiscal_year
    
    # Create date for first day of the month
    return date(calendar_year, month_num, 1)

In [44]:
def fiscal_info(year, month):
    # Fiscal year: July-June (FY25 = July 2024 - June 2025)
    fy = year + 1 if month >= 7 else year
    quarter = ((month - 7) % 12) // 3 + 1
    return f"FY{str(fy)[2:]}", f"FY{str(fy)[2:]}-Q{quarter}"

def make_entry(year, month):
    fy, quarter = fiscal_info(year, month)
    return [
        {"Literal": {"Value": f"'{fy}'"}},
        {"Literal": {"Value": f"'{quarter}'"}},
        {"Literal": {"Value": f"'{month_name[month]} {year}'"}}
    ]

def determine_fiscal_year(current_date=None, fiscal_start_month=7):
    if current_date is None:
        current_date = datetime.now()
    
    year = current_date.year
    month = current_date.month
    
    # If we're in or after the fiscal start month, we're in the next fiscal year
    if month >= fiscal_start_month:
        return year + 1
    else:
        return year

def generate_fiscal_dates(incremental=False):
    """Generate fiscal dates. FY starts in July."""
    now = datetime.now()
    dates = []
    
    if incremental:
        # Current and previous month
        for offset in [1, 0]:  # Previous, then current
            date = datetime(now.year, now.month, 1) if offset == 0 else \
                   datetime(now.year - (now.month == 1), (now.month - 2) % 12 + 1, 1)
            dates.append(make_entry(date.year, date.month))
    else:
        year, month = determine_fiscal_year() - 2, 7
        while year < now.year or (year == now.year and month <= now.month):
            dates.append(make_entry(year, month))
            month = month % 12 + 1
            if month == 1:
                year += 1
    
    return dates

# fiscal_dates = [
#     [{"Literal": {"Value": "'FY25'"}}, {"Literal": {"Value": "'FY25-Q4'"}},{"Literal": {"Value": "'June 2025'"}}],
#     [{"Literal": {"Value": "'FY26'"}}, {"Literal": {"Value": "'FY26-Q1'"}},{"Literal": {"Value": "'July 2025'"}}],
#     [{"Literal": {"Value": "'FY26'"}}, {"Literal": {"Value": "'FY26-Q1'"}},{"Literal": {"Value": "'August 2025'"}}],
#     [{"Literal": {"Value": "'FY26'"}}, {"Literal": {"Value": "'FY26-Q1'"}},{"Literal": {"Value": "'September 2025'"}}],
# ]

In [45]:
def is_transient_error(exception):
    if isinstance(exception, requests.exceptions.HTTPError):
        # Retry only for 5xx or 429
        status = exception.response.status_code
        return status >= 500 or status == 429
    return isinstance(exception, requests.exceptions.RequestException)

@retry(retry=retry_if_exception(is_transient_error), stop=stop_after_attempt(3), wait=wait_exponential())
def fetch_data(account, fiscal_date, query) -> pd.DataFrame:
    # update query to reflect correct account + fiscal month
    query["queries"][0]["Query"]["Commands"][0]["SemanticQueryDataShapeCommand"]["Query"]["Where"][1]["Condition"]["In"]["Values"][0][0]["Literal"]["Value"] = account
    query["queries"][0]["Query"]["Commands"][0]["SemanticQueryDataShapeCommand"]["Query"]["Where"][0]["Condition"]["In"]["Values"][0] = fiscal_date

    response = requests.post(url, headers=headers, json=query)
    response.raise_for_status()
    df = extract_dm1_data(response)

    if df is None:
        account = account.replace("'", "").replace("/", "-")
        print(f"⚠️ No data found for {account} in {fiscal_date[-1]['Literal']['Value']}")
    
    return df


In [46]:
fiscal_dates = generate_fiscal_dates(incremental=incremental)
columns = [
    "TPAccountName",
    "EnrollmentCustomerName",
    "SubscriptionGUID",
    "SubscriptionName",
    "FiscalMonth", 
    "$ Organic ACR", 
    "$ Average Daily Organic ACR",
    "ServiceLevel1",
    "ServiceLevel2",
    "ServiceLevel4",
    "FiscalYear",
]

with ThreadPoolExecutor() as executor:
    for account in accounts:
        dfs = []
        futures = {executor.submit(fetch_data, account, fiscal_date, query): fiscal_date for fiscal_date in fiscal_dates}
        for future in as_completed(futures):
            fiscal_date = futures[future]
            try:
                result = future.result()
            except Exception as e:
                print(f"❌ Error fetching data for {account} in {fiscal_date[-1]['Literal']['Value']}: {e}")
                for f in futures:
                    f.cancel()

            if result is not None:
                dfs.append(result)

        # Combine all dataframes for the account
        if dfs:
            account = account.replace("'", "").replace("/", "-")
            df = pd.concat(dfs, ignore_index=True)
            df = df[columns] # makes sure that the columns are in the right order.
            df["FiscalMonth"] = df["FiscalMonth"].apply(parse_fiscal_month)
            df = df.sort_values("FiscalMonth", ascending=True).reset_index(drop=True)
            df.to_parquet(f"/lakehouse/default/Files/acr_{account}.parquet", index=False)
            df.to_excel(f"/lakehouse/default/Files/acr_{account}.xlsx", index=False)
            print(f"✅ Successfully retrieved data for >> {account} <<")


✅ Successfully retrieved data for >> bpost <<
✅ Successfully retrieved data for >> Eurocontrol <<
✅ Successfully retrieved data for >> FOD BOSA <<
✅ Successfully retrieved data for >> Nationale Loterij-Loterie Nationale <<
✅ Successfully retrieved data for >> Region Bruxelles Capitale <<
✅ Successfully retrieved data for >> Vlaamse Overheid <<
